In [ ]:



for date in sample_dates:
    date_data = data.xs(date, level='date')

    date_factors = []
    date_returns = []


    for stock_id in date_data.index:
        stock_data = data.loc[:date]
        stock_data = stock_data.xs(stock_id, level='stock')


        factor_series = self.evaluate_tree(tree, stock_data, stock_id)
        factor_value = float(factor_series.iloc[-1])

        forward_return = stock_data['returns_5d'].iloc[-1] if len(stock_data) > 0 else np.nan

        if np.isfinite(factor_value) and np.isfinite(forward_return):
            date_factors.append(factor_value)
            date_returns.append(forward_return)

            # ✅ 新增：记录当日 coverage（在 ic 判断前记录，避免只挑“容易算出ic”的日子）
            coverage = len(date_factors) / total_stocks
            coverages.append(coverage)

            if len(date_factors) > 10:
                ic = self.calculate_ic(np.array(date_factors), np.array(date_returns))
                if np.isfinite(ic):
                    all_ics.append(float(ic))
